# 医療テキスト分析システム

このノートブックでは以下の機能を実行できます：
1. ダミーデータの生成
2. LLMサーバーの起動確認
3. テキスト分析の実行
4. 結果の確認と保存

In [7]:
import sys
import os
import pandas as pd
import requests
from IPython.display import display
from openai import OpenAI

# srcディレクトリをパスに追加
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.data.data_generator import MedicalDataGenerator
from src.analyzer.excel_analyzer import ExcelAnalyzer

## 1. LLMサーバーの起動確認

**注意**: 別ターミナルで `python src/analyzer/llm_server.py` を実行してサーバーを起動しておく必要があります。

In [2]:
def check_llm_server():
    server_url = "http://localhost:8000"
    try:
        # ヘルスチェックエンドポイントを確認
        health_response = requests.get(f"{server_url}/health", timeout=5)
        if health_response.status_code == 200:
            print("✅ LLMサーバーが正常に動作しています")
            
            # APIエンドポイントも確認
            api_response = requests.get(f"{server_url}/v1/chat/completions", timeout=5)
            if api_response.status_code == 405:  # POSTメソッドのみ許可される場合
                print("✅ APIエンドポイントも利用可能です")
                return True
            else:
                print("⚠️ APIエンドポイントが応答しませんでした")
                print(f"ステータスコード: {api_response.status_code}")
                return False
    except requests.exceptions.ConnectionError:
        print("❌ LLMサーバーに接続できません")
        print(f"サーバーURL: {server_url}")
        print("\n以下を確認してください：")
        print("1. サーバーが起動しているか")
        print("2. URLが正しいか")
        print("3. ファイアウォールの設定")
        print("\nサーバー起動コマンド：")
        print("python src/analyzer/llm_server.py")
    except requests.exceptions.Timeout:
        print("❌ サーバーの応答がタイムアウトしました")
    except Exception as e:
        print(f"❌ 予期せぬエラーが発生しました: {str(e)}")
    return False

# サーバーの状態を確認
check_llm_server()

✅ LLMサーバーが正常に動作しています
✅ APIエンドポイントも利用可能です


True

### テスト呼び出し

In [3]:
# OpenAIクライアントの初期化
client = OpenAI(
    api_key="EMPTY",
    base_url="http://localhost:8000/v1"
)

try:
    # テスト呼び出し
    completion = client.completions.create(
        model="Qwen/Qwen2.5-72B-Instruct-GPTQ-Int4",
        prompt="San Francisco is a"
    )
    print("✅ LLMサーバーのテスト呼び出しに成功しました")
    print("応答結果:", completion)
except Exception as e:
    print("❌ LLMサーバーのテスト呼び出しに失敗しました")
    print(f"エラー内容: {str(e)}")


✅ LLMサーバーのテスト呼び出しに成功しました
応答結果: Completion(id='cmpl-b46586835c364f9e95776c40a247dd56', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=None, text=' rock supergroup that formed in San Francisco, California, in 201', stop_reason=None, prompt_logprobs=None)], created=1739449171, model='Qwen/Qwen2.5-72B-Instruct-GPTQ-Int4', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=4, total_tokens=20, completion_tokens_details=None, prompt_tokens_details=None))


## 2. テキスト分析の実行

生成したデータに対して、各種分析を実行します。

In [8]:
# アナライザーの初期化
try:
    # データディレクトリの作成（存在しない場合）
    os.makedirs("data", exist_ok=True)
    
    # ファイルパスの設定
    data_file = "data/sample_data.xlsx"
    template_file = "templates/prompt_templates.json"
    
    analyzer = ExcelAnalyzer(
        llm_server_url="http://localhost:8000/v1",
        template_path=template_file
    )
    
    # ファイルの読み込み
    if not analyzer.load_excel(data_file):
        print("❌ ファイルの読み込みに失敗しました")
    else:
        print("✅ ファイルの読み込みが完了しました")
        analyzer.display_data_info()
except Exception as e:
    print(f"❌ エラーが発生しました: {str(e)}")
    print("以下を確認してください：")
    print(f"1. {template_file} ファイルが存在すること")
    print(f"2. {data_file} ファイルが存在すること")
    print("3. LLMサーバーが起動していること")

# 各種分析の実行
analysis_templates = [
    "cancer_diagnosis",   # がん診断名の抽出
    #"cancer_stage",       # ステージ情報の抽出
    #"diagnostic_test",    # 診断検査の抽出
    #"first_treatment",    # 初回治療情報の抽出
    #"chemotherapy_info",  # 抗がん剤治療情報の抽出
    #"surgery_type",       # 術式の抽出
    #"special_notes"       # 特記事項の抽出
]

# 分析の実行
for template in analysis_templates:
    print(f"\n分析実行中: {template}")
    if not analyzer.analyze_with_template(template):
        print(f"警告: {template}の分析に失敗しました")

# 結果の保存
analyzer.save_results("data/analyzed_results.xlsx")

テンプレートを読み込みました（7件）
ファイルの読み込みが完了しました
✅ ファイルの読み込みが完了しました
データ行数: 489

列一覧:
- ID
- day
- text

分析実行中: cancer_diagnosis
分析が完了しました。新しい列 '分析結果_がん診断名抽出' が追加されました。
分析結果を 'data/analyzed_results.xlsx' に保存しました

分析結果の概要:
- 分析結果_がん診断名抽出:
  総データ数: 489
  ユニークな値の数: 24
  未検出(N/A)の数: 0
  主な抽出結果:
    - "子宮体部類内膜癌": 77件
    - "子宮頸がん": 38件
    - "卵巣がん": 37件
    - "頚部腺癌": 36件
    - "子宮頸癌": 35件


True

## 3. 分析結果の確認と保存

In [9]:
# 分析結果の保存
output_path = "sample_data_analyzed.xlsx"
analyzer.save_results(output_path)

# 結果の確認
results_df = pd.read_excel(output_path)
print("\n分析結果のプレビュー:")
display(results_df.head())

分析結果を 'sample_data_analyzed.xlsx' に保存しました

分析結果の概要:
- 分析結果_がん診断名抽出:
  総データ数: 489
  ユニークな値の数: 24
  未検出(N/A)の数: 0
  主な抽出結果:
    - "子宮体部類内膜癌": 77件
    - "子宮頸がん": 38件
    - "卵巣がん": 37件
    - "頚部腺癌": 36件
    - "子宮頸癌": 35件

分析結果のプレビュー:


,ID,day,text,分析結果_がん診断名抽出
0,1,2023-01-01,組織診の結果、体部類内膜癌、Stage IVBと診断。 抗凝固薬（イグザレルト）服用中,"""子宮体部類内膜癌"""
1,1,2023-01-11,治療方針：腹腔鏡下子宮全摘の方針。,"""子宮体部類内膜癌"""
2,1,2023-02-10,手術記録：腹腔鏡下子宮全摘出術施行。手術時間187分、出血量214ml。,"""子宮体部類内膜癌"""
3,1,2023-02-20,術後化学療法としてドセタキセル+カルボプラチンを開始。 抗凝固薬（イグザレルト）服用中,"""子宮体部類内膜癌"""
4,1,2023-03-05,術後化学療法としてドセタキセル+カルボプラチンを開始。 糖尿病（HbA1c 9.1%）あり,"""子宮体部類内膜癌"""


## 4. 特定の患者の詳細確認

In [10]:
def display_patient_details(df, patient_id):
    patient_data = df[df['ID'] == patient_id].sort_values('day')
    print(f"患者ID {patient_id} の経過:")
    
    for _, row in patient_data.iterrows():
        print(f"\n日付: {row['day']}")
        print(f"記録: {row['text']}")
        
        # 分析結果列の表示
        analysis_columns = [col for col in df.columns if col.startswith('分析結果_')]
        for col in analysis_columns:
            if row[col] != 'N/A':
                print(f"{col}: {row[col]}")

# 最初の患者IDについて詳細を表示
first_patient_id = results_df['ID'].iloc[0]
display_patient_details(results_df, first_patient_id)

患者ID 1 の経過:

日付: 2023-01-01 00:00:00
記録: 組織診の結果、体部類内膜癌、Stage IVBと診断。 抗凝固薬（イグザレルト）服用中
分析結果_がん診断名抽出: "子宮体部類内膜癌"

日付: 2023-01-11 00:00:00
記録: 治療方針：腹腔鏡下子宮全摘の方針。
分析結果_がん診断名抽出: "子宮体部類内膜癌"

日付: 2023-02-10 00:00:00
記録: 手術記録：腹腔鏡下子宮全摘出術施行。手術時間187分、出血量214ml。
分析結果_がん診断名抽出: "子宮体部類内膜癌"

日付: 2023-02-20 00:00:00
記録: 術後化学療法としてドセタキセル+カルボプラチンを開始。 抗凝固薬（イグザレルト）服用中
分析結果_がん診断名抽出: "子宮体部類内膜癌"

日付: 2023-03-05 00:00:00
記録: 術後化学療法としてドセタキセル+カルボプラチンを開始。 糖尿病（HbA1c 9.1%）あり
分析結果_がん診断名抽出: "子宮体部類内膜癌"

日付: 2023-03-13 00:00:00
記録: 術後化学療法として毎週パクリタキセルを開始。 心機能低下（EF 55%）あり
分析結果_がん診断名抽出: "子宮体部類内膜癌"
